In [1]:
import os
import json
import fnmatch
import glob
import bids
import shutil
import multiprocessing
import pandas as pd
from bids import BIDSLayout
from joblib import Parallel, delayed

derivatives = "/Volumes/data/prebiostress/data/derivatives/"
layout = BIDSLayout(derivatives,validate=False)

grouping = pd.read_csv('participants.tsv', delimiter = '\t')

In [2]:
# do something to remove existing txt file
!rm seg_vol.csv
!echo "Subject,Group,Session,GM,WM,CSF,Total" > seg_vol.csv
count=0
for subj in layout.get_subjects():
    for ses in layout.get_sessions():
               
        targ = os.path.join(derivatives,"sub-"+subj,"ses-"+ses,"anat","fsl")
        csf = targ +"/sub-" +subj +"_ses-" +ses +"_pve_0.nii.gz"
        gm = targ +"/sub-" +subj +"_ses-" +ses +"_pve_1.nii.gz"
        wm = targ +"/sub-" +subj +"_ses-" +ses +"_pve_2.nii.gz"
        
        cmd = "fslstats " +csf +" -M -V | awk '{ print $1 * $3  }'"
        csfvol = os.popen(cmd).read()
        
        cmd = "fslstats " +gm +" -M -V | awk '{ print $1 * $3 }'"
        gmvol = os.popen(cmd).read()
        
        cmd = "fslstats " +wm +" -M -V | awk '{ print $1 * $3 }'"
        wmvol = os.popen(cmd).read()
        
        maskpath = glob.glob(targ[:-3]+"*dn_brainmask.nii.gz") # find all T1s   
        maskpath = "".join(maskpath)
        
        cmd = "fslstats " +maskpath +" -V | awk '{ print $2 }'"
        vol = os.popen(cmd).read()
        
        #print("subj,grouping.iloc[count]["group"],ses,csfvol[:-1], gmvol[:-1], wmvol[:-1],vol[:-1]")
        #os.system("echo 'subj','grouping.iloc[count]['group']'','ses','csfvol[:-1]'','gmvol[:-1]'','wmvol[:-1]'','vol[:-1]' >> seg_vol.csv")
        
        cmd= "echo sub-" +subj +"\," +grouping.iloc[count]["group"] +"\," +ses +"\," +csfvol[:-1] +"\," +gmvol[:-1] +"\," +wmvol[:-1]  +"\," +vol[:-1] +" >> seg_vol.csv"
        os.system(cmd)
    count = count+1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
